In [5]:
from bs4 import BeautifulSoup
import requests
# import time, os
import re
# import urllib
# import json
# from selenium import webdriver
# from selenium.webdriver.common.keys import Keys
import pandas as pd
from datetime import datetime
import numpy as np

In [6]:
start, end = 2274000, 2274010



In [7]:
hanum = '2268734'

In [8]:
allf = []
for id in range(start, end):
    site = 'https://get61.com/hand_detail?handid={}'
    page = requests.get(site.format(id)).text
    soup = BeautifulSoup(page, 'html5lib')

    cardre = re.compile('var play11')
    cards = soup.find(text=cardre)
    a = cards.split(';')
    a
    b = a[0].split(' ')
    rounds = [i.split(' ') for i in a]
    if rounds[41][-1] == '"NA"':
        continue
    else:

        plays = [rounds[i][3].replace('"', '') for i in range(30)]

        handa = plays[:6]
        handb = plays[6:12]
        handc = plays[12:18]
        handd = plays[18:24]
        hande = plays[24:32]

        nameslist = [row.split('=') for row in a[36:41]]
        scores = [int(rounds[i][3].replace('"', '')) for i in range(41, 46)]
        names = [nameslist[row][1].strip().replace('"', '') for row in range(5)]
        leads = [int(rounds[i][3].replace('"', '')) for i in range(30, 36)]

        blind1, blind2 = rounds[48][-1].replace('"',''), rounds[49][-1].replace('"','')
        bury1, bury2 = rounds[46][-1].replace('"',''), rounds[47][-1].replace('"','')

        call_ace = rounds[50][-1].replace('"','')
        ace_suit = rounds[53][-1].replace('"','')
        is_leaster = rounds[51][-1].replace('"','')

        player1 = [names[0]] + handa + [scores[0]]
        player2 = [names[1]] + handb + [scores[1]]
        player3 = [names[2]] + handc + [scores[2]]
        player4 = [names[3]] + handd + [scores[3]]
        player5 = [names[4]] + hande + [scores[4]]
        df = pd.DataFrame([player1, player2, player3, player4, player5], columns = ['name', 1, 2,3, 4, 5, 6, 'score'])
        
        df['handid'] = id
        df['picker'] = df['score'].apply(lambda x: 'picker' if x == max(scores, key=abs) else 'partner' if np.sign(x) == np.sign(max(scores,                                               key=abs)) else 'no')
        df['alone'] = df['picker'].apply(lambda x: 'no' if 'partner' in df.values else 'yes')
        blind1, blind2 = rounds[48][-1].replace('"',''), rounds[49][-1].replace('"','')
        df['position'] = [0, 1, 2, 3, 4]
        df['lead0'] = leads[0]
        df['lead1'] = leads[1]
        df['lead2'] = leads[2]
        df['lead3'] = leads[3]
        df['lead4'] = leads[4]
        df['lead5'] = leads[5]
        dfsc['pick_position'] = df.apply(lambda x: 0 if x['position'] == x['lead0'] else x['position'] + (5 % x['lead0']) if x['position'] < x['lead0'] else x['position'] - x['lead0'], axis=1)
        dfsc['rd0pos'] = df.apply(lambda x: 0 if x['position'] == x['lead0'] else 4 if x['lead0'] == 1 and x['position'] == 0 else x['position'] + (5 % x['lead0']) if x['position'] < x['lead0'] else x['position'] - x['lead0'], axis=1)
        dfsc['rd1pos'] = df.apply(lambda x: 0 if x['position'] == x['lead1'] else 4 if x['lead1'] == 1 and x['position'] == 0 else x['position'] + (5 % x['lead1']) if x['position'] < x['lead1'] else x['position'] - x['lead1'], axis=1)
        dfsc['rd2pos'] = df.apply(lambda x: 0 if x['position'] == x['lead2'] else 4 if x['lead2'] == 1 and x['position'] == 0 else x['position'] + (5 % x['lead2']) if x['position'] < x['lead2'] else x['position'] - x['lead2'], axis=1)
        dfsc['rd3pos'] = df.apply(lambda x: 0 if x['position'] == x['lead3'] else 4 if x['lead3'] == 1 and x['position'] == 0 else x['position'] + (5 % x['lead3']) if x['position'] < x['lead3'] else x['position'] - x['lead3'], axis=1)
        dfsc['rd4pos'] = df.apply(lambda x: 0 if x['position'] == x['lead4'] else 4 if x['lead4'] == 1 and x['position'] == 0 else x['position'] + (5 % x['lead4']) if x['position'] < x['lead4'] else x['position'] - x['lead4'], axis=1)
        dfsc['rd5pos'] = df.apply(lambda x: 0 if x['position'] == x['lead5'] else 4 if x['lead5'] == 1 and x['position'] == 0 else x['position'] + (5 % x['lead5']) if x['position'] < x['lead5'] else x['position'] - x['lead5'], axis=1)

        dfsc['blind_1'] = blind1
        dfsc['blind_2'] = blind2

        dfsc['bury1'] = bury1
        dfsc['bury2'] = bury2

        dfsc['call_ace'] = call_ace
        dfsc['ace_suit'] = ace_suit
        dfsc['leaster'] = is_leaster
        allf.append(df)

In [9]:
dftot = pd.concat(allf)

In [10]:
dftot

,name,1,2,3,4,5,6,score,handid,picker,alone
0,island,12C,14H,8D,11S,9S,10C,-2,2274001,no,no
1,Card Lady,9D,7H,13D,11D,10S,7C,2,2274001,partner,no
2,Pan fisherman,10D,10H,11H,11C,8S,9C,-2,2274001,no,no
3,rboegel,12D,14D,12S,12H,7S,7D,4,2274001,picker,no
4,spaan,14C,9H,8C,13H,13S,13C,-2,2274001,no,no
0,Dantheblitzer,9D,12C,7S,11S,12D,13S,2,2274002,picker,no
1,Rich Harms,13D,11D,11C,8D,7C,9C,1,2274002,partner,no
2,harley.dawg1957,14H,10D,8S,14D,8C,10C,-1,2274002,no,no
3,GBPFavreFan,8H,12H,9H,11H,13H,14C,-1,2274002,no,no
4,Thomas Mardak,7D,12S,9S,14S,13C,10S,-1,2274002,no,no


https://get61.com/hand_detail?handid=226800


In [106]:
#dftot.to_pickle('2268010_2268200')
#dftot.to_pickle('2268200_2268600')
#dftot.to_pickle('2268600_2270000')
#dftot.to_pickle('2270000_2272000')
#dftot.to_pickle('2272000_2274000')